# 🚀 SageMaker Lakehouse: 판매 및 프로모션 데이터 연결을 위한 ML 솔루션

## 📊 개요

이 노트북은 Amazon SageMaker Lakehouse를 활용하여 서로 다른 저장 시스템에서 가져온 판매 및 프로모션 데이터를 통합하는 통합 데이터셋을 생성하는 방법을 보여줍니다. 이렇게 생성된 테이블은 머신 러닝 모델 개발의 기반이 될 것입니다.

## 🛠️ 기술 환경                                                                                                                                                                    
우리는 다음과 같은 기능을 제공하는 Amazon SageMaker Lakehouse를 활용합니다:

- 🔄 Amazon S3 데이터 레이크와 Amazon Redshift 데이터 웨어하우스 전반에 걸친 통합 데이터 액세스
- 🔍 Apache Iceberg 호환 도구를 사용한 원활한 쿼리 기능
- 💎 데이터 복제 없음: 데이터가 있는 곳에서 직접 쿼리하여 복사본 생성 필요성 제거


## 🏢 비즈니스 맥락

분석 팀의 구성원으로서, 우리는 판매 성과와 프로모션 활동을 연관시키는 데이터셋을 준비해야 합니다. 우리의 데이터 소스는 두 부서에 걸쳐 있습니다:

- 중앙 운영 📦: Amazon S3 데이터 레이크에 저장된 판매 데이터
- 백오피스 💼: Amazon Redshift에서 관리되는 프로모션 데이터

## 🎯 목표

우리는 다음과 같은 새로운 분석 테이블을 생성할 것입니다:

1. 🔗 S3의 판매 거래 기록과 Redshift의 프로모션 데이터 통합
2. 📈 각 판매 시점에 제품 카테고리 및 지역별 활성 프로모션 수 계산
3. 🤖 머신 러닝 팀과 공유할 준비가 된 정제되고 최적화된 데이터셋 제공
    

💡 SageMaker Lakehouse의 통합 쿼리 인터페이스는 여러 저장 솔루션 작업의 복잡성을 제거하여, 데이터 액세스 메커니즘보다는 분석 요구사항에 집중할 수 있게 해줍니다.

## 🔧 설정 및 구성

필요한 라이브러리와 Spark 세션 구성으로 개발 환경을 초기화해 보겠습니다. 다음 설정은 Amazon S3에 저장된 데이터 레이크 테이블과 Amazon Redshift의 데이터 웨어하우스 테이블의 원활한 통합을 가능하게 합니다.

In [ ]:
# 필요한 라이브러리 가져오기
import boto3
import json

# 현재 AWS 계정 ID를 검색하는 함수
def get_account_id():
    sts = boto3.client('sts')
    return sts.get_caller_identity()['Account']

# 현재 AWS 계정 ID 가져오기
account_id = get_account_id()

# 다음을 포함한 Spark 세션 구성 딕셔너리 정의:
# - Iceberg 테이블 형식 지원
# - Spark SQL 확장 및 카탈로그 설정

config_dict = {
    "--datalake-formats": "iceberg",
    "--conf": f"spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.rms_federated_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.rms_federated_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.rms_federated_catalog.glue.id={account_id}:federated_redshift_catalog/enterprise_operations --conf spark.sql.catalog.rms_federated_catalog.client.region=us-east-1 --conf spark.sql.catalog.rms_federated_catalog.glue.account-id={account_id} --conf spark.sql.catalog.spark_catalog.client.region=us-east-1 --conf spark.sql.catalog.spark_catalog.glue.account-id={account_id}"
}

# 구성을 적절히 형식이 지정된 JSON 문자열로 변환
# JSON 호환성을 위해 작은따옴표를 큰따옴표로 대체
config_json = json.dumps(config_dict, indent=4).replace("'", '"')

# IPython 인터페이스를 가져와서 구성 설정
ip = get_ipython()
ip.run_cell_magic('configure', '-f --name project.spark.fineGrained', config_json)

💡 참고: 다음 셀을 실행하면 Glue 인터랙티브 세션이 초기화되며 완료하는 데 약 1분 정도 소요될 수 있습니다.

In [ ]:
%%pyspark project.spark.fineGrained
spark

## 📊 데이터 탐색: Glue 데이터 카탈로그 (S3 데이터 레이크)

S3 기반 데이터 레이크에서 Glue 기본 카탈로그를 통해 사용 가능한 데이터를 살펴보겠습니다.

기본 Glue 카탈로그에서 사용 가능한 모든 데이터베이스 나열

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"show databases").show(truncate=False)

In [ ]:
%%pyspark project.spark.fineGrained

# 프로젝트 데이터베이스 이름 가져오기
project_db = spark.sql("show databases").collect()[0]['namespace']


기본 데이터베이스의 테이블 표시

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"show tables from {project_db}").show(truncate=False)

`sales_data` 테이블 데이터 보기 (처음 10개 레코드)

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"select * from {project_db}.sales_data limit 10").show(truncate=False)

## 📊 데이터 탐색: Redshift 연합 카탈로그

이제 Redshift 데이터 웨어하우스에 저장된 데이터를 살펴보겠습니다.

Redshift에서 사용 가능한 모든 데이터베이스 나열.

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"show databases in rms_federated_catalog").show(truncate=False)

Redshift의 `public` 스키마에 있는 테이블 표시

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"show tables from rms_federated_catalog.public").show(truncate=False)

`promotions` 테이블 데이터 보기 (처음 10개 레코드)

In [ ]:
%%pyspark project.spark.fineGrained
# Redshift 연결 시 콜드 스타트 문제 처리 Redshift Serverless 의 경우 장시간 사용하지 않은 경우 시작할 때 시간이 걸리는 경우가 있어, 간단한 쿼리로 DB를 깨우는 작업을 합니다
try:
    spark.sql(f"select * from rms_federated_catalog.public.promotions limit 1").count()
except:
    pass

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"select * from rms_federated_catalog.public.promotions limit 10").show(truncate=False)

## 🔄 ML 모델링을 위한 판매 및 프로모션 데이터셋 준비

이 섹션에서는 데이터 레이크의 판매 데이터와 데이터 웨어하우스의 프로모션 정보를 결합하여 향후 머신 러닝 모델링을 위한 특성이 풍부한 데이터셋을 만듭니다. 결과 테이블에는 각 판매 제품 카테고리 및 지역에 대한 활성 프로모션 수가 포함됩니다.

보강된 데이터셋을 프로젝트 데이터베이스의 새 테이블로 저장합니다.

In [ ]:
%%pyspark project.spark.fineGrained

# 판매 데이터와 프로모션을 조인하고 판매당 활성 프로모션 계산
final_table = spark.sql(f"""
SELECT 
    s.*,
    COUNT(p.promotion_id) as active_promotions
FROM 
    {project_db}.sales_data s
LEFT JOIN 
    rms_federated_catalog.public.promotions p 
    ON s.region = p.region
    AND s.product_category = p.product_category
    AND s.order_date BETWEEN p.start_date AND p.end_date
GROUP BY 
    s.region,
    s.country,
    s.item_type,
    s.product_category,
    s.sales_channel,
    s.order_priority,
    s.order_date,
    s.order_id,
    s.ship_date,
    s.units_sold,
    s.unit_price,
    s.unit_cost,
    s.total_revenue,
    s.total_cost,
    s.total_profit
""")

# 최종 데이터셋의 임시 뷰 생성
final_table.createOrReplaceTempView("temp_final_table")

# CTAS(Create Table As Select)를 사용하여 영구 테이블 생성
spark.sql(f"""
    CREATE TABLE {project_db}.sales_table_enriched_w_campaigns
    USING PARQUET
    AS 
    SELECT * FROM temp_final_table
""")

새로 생성된 보강 테이블 미리보기

In [ ]:
%%pyspark project.spark.fineGrained
spark.sql(f"select * from {project_db}.sales_table_enriched_w_campaigns limit 10").show(truncate=False)